## Amir Kabir University of technology

### Computational Intelligence - Project 1 - 9931xx6

In [1]:
import pandas as pd
import torch
from tensor import Tensor
from torch.utils.data.dataset import Dataset
from torchsummary import summary
from tqdm import tqdm
import matplotlib.pylab as plt

In [2]:
#don't change this cell
device = "cuda" if torch.cuda.is_available() else "cpu"
device 

'cpu'

In [3]:
#do not change this cell
class CSVDataset(Dataset):
    def __init__(self, path: str):
        self.data = pd.get_dummies(pd.read_csv(path), columns=['Species']).astype(float)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        row = self.data.iloc[idx]
        features = torch.tensor(row[:-3].values, dtype=torch.float32)
        label = torch.tensor([row.iloc[-3],row.iloc[-2],row.iloc[-1]], dtype=torch.float32)
        return features, label

    @property
    def classes(parameter_list):
        return ['Iris-setosa','Iris-versicolor','Iris-virginica']

In [4]:
import os

base_path = os.getcwd()

#TODO change pathes
path_train = os.path.join(base_path, "Iris-Train.csv")
path_test = os.path.join(base_path, "Iris-Test.csv")

dataset_train = CSVDataset(path_train)
dataset_test = CSVDataset(path_test)

In [5]:
from torch.utils.data import DataLoader

#TODO set parameters
loader_train = DataLoader(
    dataset=dataset_train,
    batch_size=10,
    shuffle=True
)

loader_test = DataLoader(
    dataset=dataset_test,
    batch_size=10,
    shuffle=True
)

In [6]:
#don't change this cell
print(f"batch   size\t= {loader_train.batch_size}")
print(f"train   size\t= {len(dataset_train):,}")
print(f"test    size\t= {len(dataset_test):,}")
print(f"Class   names\t= {dataset_train.classes}")
print(f"input   shape\t= {dataset_train[0][0].shape}")
print(f"output  shape\t= {dataset_train[0][1].shape}")
print(f"example input\t= {dataset_train[0][0]}")
print(f"example output\t= {dataset_train[0][1]}")
print(f"example label\t= {dataset_train.classes[dataset_train[0][1].argmax()]}")

batch   size	= 10
train   size	= 105
test    size	= 60
Class   names	= ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
input   shape	= torch.Size([4])
output  shape	= torch.Size([3])
example input	= tensor([5.1000, 3.5000, 1.4000, 0.2000])
example output	= tensor([1., 0., 0.])
example label	= Iris-setosa


In [7]:
from myModel import MyModel

mymodel = MyModel()
MyModel.summary(self=mymodel)

linear1:  linear - total param: 20 - in: 4, out: 5
linear2:  linear - total param: 15 - in: 5, out: 3


In [13]:
from loss.mse import MeanSquaredError
from optimizer import SGD

def one_epoch_learning(
    model: MyModel,
    criterion: MeanSquaredError,
    loader: DataLoader,
    optimizer: SGD,
) -> int:
    accs = 0
    for data, label in loader:
        data: Tensor = data.to(device)
        label: Tensor = label.to(device)

        optimizer.zero_grad()
        res: Tensor = model(data)
        loss: Tensor = criterion(res, label)
        loss.backward()
        
        optimizer.step()
        accs += (res.argmax(dim=1) == label.argmax(dim=1)).sum().item()
    return accs

In [14]:
#don't change this cell
def calculate_accuracy(
    model: MyModel, loader: DataLoader, criterion: MeanSquaredError
) -> int:
    model.eval()
    accs = 0
    for data, label in loader:
        data: Tensor = data.to(device)
        label: Tensor = label.to(device)

        res: Tensor = model(data)

        accs += (res.argmax(dim=1) == label.argmax(dim=1)).sum().item()
    return accs

In [15]:
def train(
    model: MyModel,
    criterion: MeanSquaredError,
    loader_train: DataLoader,
    loader_test: DataLoader,
    optimizer: SGD,
    epoch: int,
):
    results_train = []
    results_test = []
    for i in tqdm(range(epoch)):
        res_train = one_epoch_learning(model, criterion, loader_train, optimizer)
        results_train.append(res_train / len(loader_train.dataset))
        res_test = calculate_accuracy(model, loader_test, criterion)
        results_test.append(res_test / len(loader_test.dataset))
    return results_train, results_test

In [16]:
from loss import CategoricalCrossEntropy

EPOCH = 1
optimizer = SGD(mymodel.parameters())
loss = CategoricalCrossEntropy

train_accs, test_accs = train(mymodel, loss, loader_train, loader_test, optimizer, EPOCH)

  0%|          | 0/1 [00:00<?, ?it/s]

Tensor([[0.         0.         3.45054944 3.88036634 2.39426558]
 [0.         0.         4.35972476 4.54506551 2.9507286 ]
 [0.         0.         3.19950863 3.54492573 2.18188187]
 [0.         0.         2.09196785 2.56349758 1.40217211]
 [0.         0.         2.93926827 3.39815855 2.03618272]
 [0.         0.         3.38183818 3.67007492 2.30012334]
 [0.         0.         3.39984386 3.4213491  2.2144315 ]
 [0.         0.         2.79304566 2.9826335  1.88242252]
 [0.         0.         3.56725777 4.16313335 2.50394769]
 [0.         0.         3.21934552 3.67791398 2.23549066]], requires_grad=True)
(10, 5)
Tensor([[8.55791160e-02 9.12817374e-01 1.60350966e-03]
 [6.69765690e-02 9.32367390e-01 6.56041370e-04]
 [1.04355240e-01 8.92851241e-01 2.79351848e-03]
 [1.57605385e-01 8.30908961e-01 1.14856540e-02]
 [1.07479764e-01 8.89224721e-01 3.29551474e-03]
 [1.00377370e-01 8.97247680e-01 2.37494986e-03]
 [1.24984383e-01 8.71055381e-01 3.96023590e-03]
 [1.45529695e-01 8.47480223e-01 6.990082

TypeError: unsupported operand type(s) for /: 'Tensor' and 'Tensor'

In [12]:
#don't change this cell
plt.plot(train_accs, label="Train accuracy")
plt.plot(test_accs, label="Test accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

NameError: name 'train_accs' is not defined

In [ ]:
#don't change this cell
print(f"final train acc = {train_accs[-1]:.5f}")
print(f"final test acc = {test_accs[-1]:.5f}")
print(f"max train acc = {max(train_accs):.5f}")
print(f"max test acc = {max(test_accs):.5f}")
print(f"min train acc = {min(train_accs):.5f}")
print(f"min test acc = {min(test_accs):.5f}")

final train acc = 0.97143
final test acc = 0.93333
max train acc = 0.97143
max test acc = 0.93333
min train acc = 0.69524
min test acc = 0.68333
